![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Module 3: Progressive Agents Demo

## Workshop: Context Engineering Matters

---

## Learning Objectives

By the end of this module, you will:

1. **Observe** the evolution from basic to production-ready RAG agents
2. **Compare** three progressive stages of agent development
3. **Understand** the impact of context engineering on real systems
4. **Identify** when to apply each pattern in your own projects

---

## What You'll See

This notebook demonstrates **three stages** of RAG agent development using the same course catalog:

### Stage 1: Baseline RAG (Information Overload)
- **Problem**: Returns ALL course details for every match
- **Result**: ~6,000+ tokens, overwhelming context
- **Learning**: "More information ≠ better answers"
- **Search**: Semantic (vector) search only

### Stage 2: Context-Engineered (Optimized)
- **Improvement**: Applies context engineering techniques
- **Result**: ~539 tokens (91% reduction)
- **Learning**: Clean, transform, optimize context
- **Search**: Semantic (vector) search only

### Stage 3: Hierarchical Retrieval (Production-Ready)
- **Improvement**: Progressive disclosure + intent classification
- **Result**: ~700 tokens with full syllabi for top matches
- **Learning**: Best of both worlds - efficient AND informative
- **Search**: Semantic (vector) search only

---

## 🔍 Note on Search Strategies

**These stages (1-3) use semantic search only** because they focus on demonstrating **context engineering** techniques:
- Stage 1: Information overload problem
- Stage 2: Context optimization solutions
- Stage 3: Progressive disclosure pattern

**For hybrid search (semantic + keyword)**, see:
- **Module 2**: Learn all search types (vector, keyword, hybrid)
- **Stage 4+ Agents**: Production agents with hybrid search + NER
- **Module 5**: Building agents with advanced search strategies

This progression is intentional - master context engineering first, then add search complexity!

---

## Prerequisites

- Completed Module 2: RAG Essentials
- Redis Stack running with course data loaded
- OpenAI API key configured

---

## Important Note

These agents are **pre-built** in the `/progressive_agents` directory.

**For this demo**, we'll run them directly in the notebook using their Python APIs. Each agent exposes a simple interface that we can call programmatically.

In later modules, you'll learn how to build agents like these yourself.

---

## Part 1: Setup and Verification

In [1]:
# Standard library imports
import os
import sys
from pathlib import Path

# Enable nested event loops for Jupyter
import nest_asyncio
nest_asyncio.apply()

# Verify we're in the right directory
project_root = Path.cwd().parent if Path.cwd().name == "workshop" else Path.cwd()

# Verify agents exist
stage1_path = project_root / "progressive_agents" / "stage1_baseline_rag"
stage2_path = project_root / "progressive_agents" / "stage2_context_engineered"
stage3_path = project_root / "progressive_agents" / "stage3_full_agent_without_memory"

print(f"""Setup Verification
{'=' * 60}
Project Root: {project_root}
Stage 1 Agent: {'✓ Found' if stage1_path.exists() else '✗ Missing'}
Stage 2 Agent: {'✓ Found' if stage2_path.exists() else '✗ Missing'}
Stage 3 Agent: {'✓ Found' if stage3_path.exists() else '✗ Missing'}

Note: Agents will be run directly in the notebook using their Python APIs
      Using nest_asyncio to handle async event loops in Jupyter
""")

Setup Verification
Project Root: /Users/nitin.kanukolanu/workspace/context-eng-matters
Stage 1 Agent: ✓ Found
Stage 2 Agent: ✓ Found
Stage 3 Agent: ✓ Found

Note: Agents will be run directly in the notebook using their Python APIs
      Using nest_asyncio to handle async event loops in Jupyter



---

## Part 2: Stage 1 - Baseline RAG (Information Overload)

### What This Stage Demonstrates

**Architecture:**
- Simple semantic search with Redis
- Returns FULL course details for ALL matches
- No context engineering or optimization

**The Problem:**
- Query: "What machine learning courses are available?"
- Returns: Complete details for 5 courses (~6,000 tokens)
- Includes: Full 14-week syllabi, all assignments, all prerequisites
- Result: Information overload, high cost, degraded quality

Let's run it from the command line:

In [2]:
# Import Stage 1 agent
import sys
sys.path.insert(0, str(project_root / "progressive_agents" / "stage1_baseline_rag"))

from agent import setup_agent, initialize_state

# Define the query we'll use for all stages
query = "What machine learning courses are available for beginners?"

print(f"""Running Stage 1: Baseline RAG
{'=' * 60}
Query: "{query}"

Running...
""")

# Setup and run Stage 1 agent
workflow, course_manager = setup_agent(auto_load_courses=True, verbose=False)
state = initialize_state(query)
result = workflow.invoke(state)

# Print the answer
print("\n" + "=" * 60)
print("📝 Answer:")
print("=" * 60)
print(result["final_answer"])
print()

# Print metrics
print("=" * 60)
print("📊 Metrics:")
print("=" * 60)
print(f"   Courses Found: {result['courses_found']}")
if result.get("total_tokens"):
    print(f"   Estimated Tokens: ~{result['total_tokens']}")
print()
print("⚠️  This agent uses RAW context - no optimization!")
print("   See Stage 2 for context engineering improvements")
print()



Running Stage 1: Baseline RAG
Query: "What machine learning courses are available for beginners?"

Running...

15:10:38 redisvl.index.index INFO   Index already exists, not overwriting.



📝 Answer:
Based on the course information provided, there is currently one machine learning course available:

### Course Details:
- **Course Code:** CS007
- **Title:** Machine Learning
- **Description:** Introduction to machine learning algorithms and applications, covering supervised and unsupervised learning, as well as neural networks.
- **Credits:** 4
- **Difficulty Level:** Advanced
- **Format:** Hybrid (a mix of in-person and online)
- **Department:** Computer Science
- **Instructor:** Kim Vazquez
- **Schedule:** 
  - **Days:** Monday and Wednesday
  - **Time:** 10:30 AM - 11:45 AM
  - **Location:** Engineering Building 463
- **Max Enrollment:** 97
- **Current Enrollment:** 21
- **Learning Objectives:**
  - Understand ML algorithms
  - Implement classification and regression models
  - Evaluate model performance
  - Apply ML to real-world problems

### Prerequisites:
- There are no prerequisites for this course, but please note that it is categorized as advanced, which may requ

### Stage 1 Analysis

**Observations:**
- ⚠️ Returns full details for ALL matching courses
- ⚠️ Includes complete 14-week syllabi for every course
- ⚠️ High token usage (~6,000+ tokens)
- ⚠️ Information overload makes it hard to compare courses

**Key Learning:**
> "More information ≠ better answers. Context engineering is about providing the RIGHT information, not ALL information."

---

## Part 3: Stage 2 - Context-Engineered (Optimized)

### What This Stage Demonstrates

**Architecture:**
- Same semantic search as Stage 1
- **NEW**: Context engineering techniques applied
- Cleans, transforms, and optimizes retrieved data

**Context Engineering Techniques:**
1. **Cleaning**: Remove noise fields (IDs, timestamps, enrollment data)
2. **Transformation**: Convert JSON → natural text format
3. **Optimization**: Efficient token usage while preserving information

**Expected Result:**
- 91% token reduction (~539 tokens vs ~6,133)
- Better LLM understanding (natural text vs JSON)
- Trade-off: Lost syllabi to fit budget

Let's run Stage 2:

In [3]:
# Import Stage 2 agent
sys.path.insert(0, str(project_root / "progressive_agents" / "stage2_context_engineered"))

from agent import setup_agent as setup_agent_stage2, initialize_state as initialize_state_stage2

print(f"""Running Stage 2: Context-Engineered
{'=' * 60}
Query: "{query}"

Running...
""")

# Setup and run Stage 2 agent
workflow2, course_manager2 = setup_agent_stage2(auto_load_courses=True, verbose=False)
state2 = initialize_state_stage2(query)
result2 = workflow2.invoke(state2)

# Print the answer
print("\n" + "=" * 60)
print("📝 Answer:")
print("=" * 60)
print(result2["final_answer"])
print()

# Print metrics
print("=" * 60)
print("📊 Metrics:")
print("=" * 60)
print(f"   Courses Found: {result2['courses_found']}")
if result2.get("total_tokens"):
    print(f"   Estimated Tokens: ~{result2['total_tokens']}")
    if result.get("total_tokens"):
        reduction = ((result['total_tokens'] - result2['total_tokens']) / result['total_tokens']) * 100
        print(f"   Token Reduction: {reduction:.1f}% vs Stage 1")
print()
print("✅ This agent uses CONTEXT ENGINEERING!")
print("   Clean, transformed, optimized context")
print()

Running Stage 2: Context-Engineered
Query: "What machine learning courses are available for beginners?"

Running...




📝 Answer:
Based on the course information provided, there is currently one machine learning course available:

### Course Details:
- **Course Code:** CS007
- **Title:** Machine Learning
- **Description:** Introduction to machine learning algorithms and applications, covering supervised and unsupervised learning, as well as neural networks.
- **Credits:** 4
- **Difficulty Level:** Advanced
- **Format:** Hybrid (a mix of in-person and online)
- **Department:** Computer Science
- **Instructor:** Kim Vazquez
- **Schedule:** 
  - **Days:** Monday and Wednesday
  - **Time:** 10:30 AM - 11:45 AM
  - **Location:** Engineering Building 463
- **Max Enrollment:** 97
- **Current Enrollment:** 21
- **Learning Objectives:**
  - Understand ML algorithms
  - Implement classification and regression models
  - Evaluate model performance
  - Apply ML to real-world problems

### Prerequisites:
- There are no prerequisites listed for this course, but please note that it is categorized as advanced, which m

### Stage 2 Analysis

**Improvements over Stage 1:**
- ✅ 91% token reduction (from ~6,133 to ~539 tokens)
- ✅ Natural text format (easier for LLM to parse)
- ✅ Removed noise fields (cleaner context)
- ✅ Same functionality, better efficiency

**Trade-offs:**
- ⚠️ Lost detailed syllabi to fit token budget
- ⚠️ Less comprehensive information per course

**Key Learning:**
> "Context engineering can dramatically reduce tokens while maintaining quality. But there's still room for improvement."

---

## Part 4: Stage 3 - Hierarchical Retrieval (Production-Ready)

### What This Stage Demonstrates

**Architecture:**
- **NEW**: Two-tier hierarchical retrieval
- **NEW**: Progressive disclosure (summaries → details on-demand)
- **NEW**: Intent classification (greeting, overview, details)
- Context engineering from Stage 2

**How It Works:**
1. Classify query intent (overview vs. detailed)
2. Search summaries (lightweight, ~60 tokens each)
3. Fetch details for top matches only (~500 tokens each)
4. Adaptive retrieval based on query type

**Expected Result:**
- ~700 tokens with syllabi for top 2-3 matches
- Best of both worlds: efficient + informative
- Production-ready architecture

Let's run Stage 3:

In [4]:
# Import Stage 3 agent
sys.path.insert(0, str(project_root / "progressive_agents" / "stage3_full_agent_without_memory"))

from agent import setup_agent as setup_agent_stage3, initialize_state as initialize_state_stage3

print(f"""Running Stage 3: Hierarchical Retrieval
{'=' * 60}
Query: "{query}"

Running...
""")

# Setup and run Stage 3 agent
workflow3, course_manager3 = setup_agent_stage3(auto_load_courses=True, verbose=False)
state3 = initialize_state_stage3(query)
result3 = workflow3.invoke(state3)

# Print the answer
print("\n" + "=" * 60)
print("📝 Answer:")
print("=" * 60)
print(result3["final_answer"])
print()

# Print metrics
print("=" * 60)
print("📊 Metrics:")
print("=" * 60)
print(f"   Courses Found: {result3['courses_found']}")
if result3.get("total_tokens"):
    print(f"   Estimated Tokens: ~{result3['total_tokens']}")
    if result.get("total_tokens"):
        reduction = ((result['total_tokens'] - result3['total_tokens']) / result['total_tokens']) * 100
        print(f"   Token Reduction: {reduction:.1f}% vs Stage 1")
print()
print("✅ This agent uses HIERARCHICAL RETRIEVAL!")
print("   Progressive disclosure + intent classification")
print()


Running Stage 3: Hierarchical Retrieval
Query: "What machine learning courses are available for beginners?"

Running...




📝 Answer:
Based on the course information provided, there is currently one machine learning course available:

### Course Details:
- **Course Code:** CS007
- **Title:** Machine Learning
- **Description:** Introduction to machine learning algorithms and applications, covering supervised and unsupervised learning, as well as neural networks.
- **Credits:** 4
- **Difficulty Level:** Advanced
- **Format:** Hybrid (a mix of in-person and online)
- **Department:** Computer Science
- **Instructor:** Kim Vazquez
- **Schedule:** 
  - **Days:** Monday and Wednesday
  - **Time:** 10:30 AM - 11:45 AM
  - **Location:** Engineering Building 463
- **Max Enrollment:** 97
- **Current Enrollment:** 21
- **Learning Objectives:**
  - Understand ML algorithms
  - Implement classification and regression models
  - Evaluate model performance
  - Apply ML to real-world problems

### Prerequisites:
- There are no prerequisites for this course, but please note that it is categorized as advanced, which may requ

### Stage 3 Analysis

**Improvements over Stage 2:**
- ✅ Includes syllabi for top matches (~700 tokens)
- ✅ Progressive disclosure (summaries → details)
- ✅ Intent classification (adapts to query type)
- ✅ Production-ready architecture

**Best of Both Worlds:**
- Efficient like Stage 2 (low token usage)
- Informative like Stage 1 (includes syllabi)
- Adaptive (changes behavior based on query)

**Key Learning:**
> "Hierarchical retrieval with progressive disclosure is the production-ready pattern. It balances efficiency with information depth."

---

## Part 5: Side-by-Side Comparison

Let's compare all three stages:

In [5]:
import pandas as pd

# Create comparison table (based on typical outputs)
comparison_data = {
    'Stage': ['Stage 1: Baseline', 'Stage 2: Context-Engineered', 'Stage 3: Hierarchical'],
    'Architecture': [
        'Flat retrieval',
        'Flat + context engineering',
        'Hierarchical + progressive disclosure'
    ],
    'Typical Tokens': [
        '~6,133',
        '~539',
        '~700'
    ],
    'Includes Syllabi': ['✓ All courses', '✗ None', '✓ Top matches'],
    'Best For': [
        'Prototyping',
        'Token-constrained systems',
        'Production systems'
    ]
}

df = pd.DataFrame(comparison_data)

print(f"""Progressive Agents Comparison
{'=' * 80}
""")
print(df.to_string(index=False))
print(f"\n{'=' * 80}")

Progressive Agents Comparison

                      Stage                          Architecture Typical Tokens Includes Syllabi                  Best For
          Stage 1: Baseline                        Flat retrieval         ~6,133    ✓ All courses               Prototyping
Stage 2: Context-Engineered            Flat + context engineering           ~539           ✗ None Token-constrained systems
      Stage 3: Hierarchical Hierarchical + progressive disclosure           ~700    ✓ Top matches        Production systems



### Visual Comparison: Token Usage

```
Stage 1: Baseline RAG
████████████████████████████████████████████████████  ~6,133 tokens
Problem: Information overload

Stage 2: Context-Engineered
████  ~539 tokens (91% reduction)
Problem: Lost syllabi

Stage 3: Hierarchical Retrieval
█████  ~700 tokens (89% reduction, WITH syllabi)
Solution: Best of both worlds ✓
```

---

## Part 6: Key Takeaways

### The Evolution of RAG Systems

1. **Stage 1 → Stage 2**: Context engineering reduces tokens by 91%
   - Cleaning, transformation, optimization
   - Trade-off: Lost detailed information

2. **Stage 2 → Stage 3**: Hierarchical retrieval adds back details efficiently
   - Progressive disclosure (summaries → details)
   - Intent classification (adaptive behavior)
   - Production-ready architecture

### When to Use Each Pattern

| Pattern | Use When | Avoid When |
|---------|----------|------------|
| **Stage 1: Baseline** | Prototyping, small datasets | Production, large datasets |
| **Stage 2: Context-Engineered** | Strict token budgets, simple queries | Need detailed information |
| **Stage 3: Hierarchical** | Production systems, complex queries | Simple prototypes |

### What You Learned

✅ **Observed** the evolution from basic to production-ready RAG
✅ **Compared** three progressive stages with the same query
✅ **Understood** the impact of context engineering on real systems
✅ **Identified** when to apply each pattern in your projects

---

## Next Steps

### Module 4: Building Your Own Agent (Coming Soon)

Now that you've seen these patterns in action, you'll learn how to build them yourself:

1. **LangGraph Basics**: State machines, nodes, edges
2. **Intent Classification**: Detecting query types
3. **Progressive Disclosure**: Implementing two-tier retrieval
4. **Memory Systems**: Adding conversation history (Stage 4)

### Explore the Code

Want to see how these agents work under the hood?

```bash
# Stage 1: Baseline RAG
progressive_agents/stage1_baseline_rag/
├── agent/
│   ├── __init__.py          # Agent setup
│   ├── workflow.py          # LangGraph workflow
│   └── nodes.py             # Workflow nodes
└── cli.py                   # Command-line interface

# Stage 2: Context-Engineered
progressive_agents/stage2_context_engineered/
├── agent/
│   ├── context_engineering.py  # NEW: Context optimization
│   └── ...
└── cli.py

# Stage 3: Hierarchical Retrieval
progressive_agents/stage3_full_agent_without_memory/
├── agent/
│   ├── intent_classifier.py    # NEW: Intent detection
│   ├── research_planner.py     # NEW: Query planning
│   └── ...
└── cli.py
```

### Try It Yourself

Run the agents from the command line:

```bash
# Stage 1
uv run python -m progressive_agents.stage1_baseline_rag.cli

# Stage 2
uv run python -m progressive_agents.stage2_context_engineered.cli

# Stage 3
uv run python -m progressive_agents.stage3_full_agent_without_memory.cli
```

---

## 🚀 What's Next: Advanced Search Strategies

The stages you've seen (1-3) use **semantic search only** to focus on context engineering fundamentals.

**Ready for production-grade search?** The advanced stages add:

### Stage 4: Hybrid Search + Named Entity Recognition
- **Combines**: Semantic search + exact matching + metadata filters
- **Features**:
  - Detects course codes in queries ("Tell me about CS101")
  - Uses Tag filters for exact matches
  - Falls back to semantic search for general queries
- **Location**: `progressive_agents/stage4_hybrid_search/`

### Stage 5+: Working Memory + Long-term Memory
- **Adds**: Agent Memory Server integration
- **Features**:
  - Remembers user preferences (semantic search)
  - Tracks conversation history
  - Combines memory + course search
- **Location**: `progressive_agents/stage5_working_memory/`

### Why This Progression?

```
Module 2: Learn search types (vector, keyword, hybrid)
    ↓
Module 3: Master context engineering (semantic only)
    ↓
Module 4: Add memory systems (semantic for memories)
    ↓
Module 5: Production agents (hybrid search + memory)
```

**Each layer builds on the previous one!**

---

## Congratulations! 🎉

You've completed Module 3: Progressive Agents Demo. You've seen how context engineering evolves from basic retrieval to production-ready systems.

**Key Insights:**
> "Context engineering is not a one-time optimization. It's an iterative process of understanding trade-offs and choosing the right architecture for your use case."

> "Different components need different search strategies. Semantic search for context engineering, hybrid search for production catalogs, no search for reference data."
